In [1]:
%%HTML
<script>
// delayed discounting
function SV_DD(stimulus_values, params) {
    var objective_value = stimulus_values[0];
    var d_days_delayed = stimulus_values[1];
    var value_fixed = stimulus_values[2];
    
    var alpha_subject_risk_aversion = params[0];
    var kappa_discount_rate = params[1];
    
    var sv_reward = Math.pow(objective_value, alpha_subject_risk_aversion) / (1 + kappa_discount_rate*d_days_delayed);
    var sv_null = Math.pow(value_fixed, alpha_subject_risk_aversion);

    var gamma = params[2];
    prob = 1. / (1 + Math.exp(-gamma * (sv_reward - sv_null)));
    return prob;
}
</script>

In [ ]:
%%HTML
<script>
// gains value space
var value_space = [7, 14, 21, 30, 41, 50, 65];
var time_reward_space = [5, 10, 30, 60, 90, 150];
var value_fixed = 10;


// participant parameter space
var alpha_space = arange_round(0., 3.1,  .1);
// discount rate
var kappa_space = [1e-05, 1.2648552168552958e-05, 1.5998587196060572e-05, 2.0235896477251556e-05, 2.559547922699533e-05, 3.237457542817647e-05, 4.094915062380427e-05, 5.1794746792312125e-05, 6.55128556859551e-05, 8.286427728546843e-05, 0.00010481131341546853, 0.00013257113655901095, 0.00016768329368110083, 0.00021209508879201905, 0.0002682695795279727, 0.000339322177189533, 0.00042919342601287783, 0.0005428675439323859, 0.0006866488450042998, 0.000868511373751353, 0.0010985411419875584, 0.0013894954943731374, 0.001757510624854793, 0.0022229964825261957, 0.002811768697974231, 0.0035564803062231283, 0.004498432668969444, 0.005689866029018299, 0.0071968567300115215, 0.009102981779915217, 0.011513953993264481, 0.014563484775012445, 0.018420699693267165, 0.023299518105153717, 0.029470517025518096, 0.037275937203149416, 0.04714866363457394, 0.05963623316594643, 0.07543120063354623, 0.09540954763499944, 0.1206792640639329, 0.15264179671752334, 0.19306977288832497, 0.244205309454865, 0.30888435964774846, 0.3906939937054621, 0.49417133613238384, 0.6250551925273976, 0.7906043210907702, 1.0];
//var kappa_space = arange_round(0,1.01,.05);

// affects nothing practically speaking
var gamma_space =  arange_round(0.5, 5.1, .5);

var stim_space = [];
var i,j;
for (i=0; i < value_space.length; i++) {
    for (j=0; j < time_reward_space.length; j++) {
        stim_space.push([value_space[i], time_reward_space[j], value_fixed]);
    }
}




var params = [alpha_space, kappa_space, gamma_space];


var contrast_idx;
var response = true;
</script>

In [ ]:
%%HTML
<div id=paramsDD></div>
<div id=alphaDD></div>
<div id=kappaDD></div>
<div id=gammaDD></div>
<script>
var func = SV_DD;
// subjective value decision?
function draw_from(stimulus_value, params, func){
    var prob = func(stimulus_value, params);
    if (Math.random() < prob)
        return 1;
    else
        return 0;
}

var response;
// # uses the contrast_idx instead of contrast for speed reasons.
// # doesn't ever estimate the contrast
contrast_idx = parseInt(Math.random()*stim_space.length);//parseInt(stim_space.length/2);

// # the estimated parameters (alpha, kappa, gamma)
var true_params = [.5, .7, 2.0];
var q = new QuestPlus(stim_space, params, func);
// # 20 trials
for (var i=0; i < 20; i++) {
    t0 = performance.now()
    response = draw_from(stim_space[contrast_idx], true_params, func);
    // # update the posterior
    q.update(contrast_idx, response);
    // # find the next contrast index
    contrast_idx=q.next_contrast();
    t1 = performance.now()
    console.log(i, stim_space[contrast_idx], q.get_fit_params(), t1-t0);
}
var fit_weighted = weighted_sum(q)
console.log(fit_weighted, "weighted sum");

var fit = q.get_fit_params();
console.log(fit, contrast_idx);
document.getElementById("paramsDD").innerHTML = "Params " +String(fit);
document.getElementById("alphaDD").innerHTML = "Estimated Alpha ".concat(fit[0]," vs True Alpha ", true_params[0]);
document.getElementById("kappaDD").innerHTML = "Estimated Kappa ".concat(fit[1]," vs True Kappa ", true_params[1]);
document.getElementById("gammaDD").innerHTML = "Estimated Gamma ".concat(fit[2]," vs True Gamma ", true_params[2]);

</script>

# Schedule Gen

In [ ]:
%%HTML
<script>
var amt_immed_options, amt_del_options, V_max, V_min, max_time, min_time;
var Amp, num_steps, d0, time_options, time_options_1, i, l;
var kappa_value = fit[1];
kappa_value = 0.002222;
</script>

In [ ]:
%%HTML
<script>
////Declare the values of immediate values, delayed values, and wait times that we are going to use

amt_immed_options = [2, 5, 10, 15, 18, 20, 27, 30, 34, 35, 41, 43, 44, 47, 51, 62, 63, 64]
amt_del_options= [5.0,10.0, 20.0, 35.0, 50.0, 65.0] //Possible immediate options

V_max= 65.0 //Maximum pay
V_min = 2.0 //Minimum pay
max_time = 180 //Longest delay
min_time = 2 //shortest delay 

//Parameter to modulate how many intermediate trials we want

Amp = 1/5.0 //The fraction of SVmax to match the final time_option
if (kappa_value < 0.003)
    Amp = 1/4.0

//Calculate the time options
num_steps = 5; //How many delayed options we want

//Longest possible delay to sample from to equally sample SVdelta pos and neg

d0 = (1/Amp -1)/kappa_value ;
if (d0 > max_time)
    d0 = max_time; //To ensure 180 days is the max limit for peopke with low kappa_value

//Initialize the time_options_1 list with time options that we want always to be present
time_options_1 = [7,15]

if (d0 < max_time) {
    num_steps = 3
    time_options_1.push(...[d0+((max_time-d0)/2),max_time]); //To ensure that the longest delays are always included for participant's with high kappa
}
//loop through the number of time_options (num_steps)
l = arange_round(0,num_steps+1,1)
for (i=0; i < l.length; i++) {
    if (i >0)
        time_options_1.push(i*(d0/num_steps)) 
    else
        time_options_1.push(min_time)
}

time_options = [];
time_options.push(...time_options_1);

amt_immed_options = amt_immed_options.sort((a,b) => a < b)
amt_del_options = amt_del_options.sort((a,b) => a < b)
time_options = time_options.sort((a,b) => a < b)
console.log('amt_immed_options',amt_immed_options)
console.log('amt_del_options',amt_del_options)
console.log('time_options',time_options)
</script>

In [ ]:
%%HTML
<script>
var _pj;
function _pj_snippets(container) {
    function in_es6(left, right) {
        if (((right instanceof Array) || ((typeof right) === "string"))) {
            return (right.indexOf(left) > (- 1));
        } else {
            if (((right instanceof Map) || (right instanceof Set) || (right instanceof WeakMap) || (right instanceof WeakSet))) {
                return right.has(left);
            } else {
                return (left in right);
            }
        }
    }
    container["in_es6"] = in_es6;
    return container;
}
_pj = {};
_pj_snippets(_pj);
keys_order = ["category", "cdd_immed_amt", "cdd_delay_amt", "cdd_delay_wait", "delay_SV", "deltaSV"];
function order_keys(dict, keys_order) {
    var ordered_dict;
    ordered_dict = {};
    for (var key, _pj_c = 0, _pj_a = keys_order, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        key = _pj_a[_pj_c];
        if (_pj.in_es6(key, dict)) {
            ordered_dict[key] = dict[key];
        }
    }
    return ordered_dict;
}
function flatten(lst) {
    /* Flatten a list of lists. */
    return lst.flatten();
}
function round_to_closest_half(number) {
    var rounded_number;
    rounded_number = (Math.round((number * 2)) / 2);
    return ((rounded_number === 0) ? 0.5 : rounded_number);
}
function zippy(x, y) {
    var zipper;
    zipper = [];
    for (var i = 0, _pj_a = x.length; (i < _pj_a); i += 1) {
        zipper.push({"x": x[i], "y": y[i]});
    }
    return zipper;
}
function mighty_numeration(iterable, start = 0) {
    var n;
    n = start;
    var arr = [];
    for (var elem, _pj_c = 0, _pj_a = iterable, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        elem = _pj_a[_pj_c];
        arr.push([n, elem]);
        n += 1;
    }
    return arr;
}
function value_repeat(value, repetitions) {
    var tmp = [];
    for (var i=0; i < repetitions; i++) {
        tmp.push(value);
    }
    return tmp;
}
</script>

In [ ]:
%%HTML
<script>
function add_SV(trials, kappa_value) {
    /*
    Adds SV and delta SV to a trials dictionary
    */
    var D, svdel, svimmed, v, zip_data, zip_data2;
    trials["delay_SV"] = [];
    trials["deltaSV"] = [];
    zip_data = zippy(trials["cdd_delay_amt"], trials["cdd_delay_wait"]);
    for (var data, _pj_c = 0, _pj_a = zip_data, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        data = _pj_a[_pj_c];
        v = data["x"];
        D = data["y"];
        trials["delay_SV"].push((v / (1 + (kappa_value * D))));
    }
    zip_data2 = zippy(trials["delay_SV"], trials["cdd_immed_amt"]);
    for (var data2, _pj_c = 0, _pj_a = zip_data2, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        data2 = _pj_a[_pj_c];
        svdel = data2["x"];
        svimmed = data2["y"];
        trials["deltaSV"].push((svdel - svimmed));
    }
    return trials;
}
function get_SE(kappa_value, amt_del_options, time_options) {
    /*
    Creates the trials of subjective equality.
    Inputs:
    - kappa_value.
    - amt_del_options: List of delayed amounts.
    - time_options: List of delay times.
    Output:
    - SE_trials: A dictionary containing immediate options with the same subjective value as delayed options.
    */
    var SE_trials, cdd_immed_amt_for_D;
    SE_trials = {"category": [], "cdd_delay_wait": [], "cdd_immed_amt": [], "cdd_delay_amt": []};
    amt_del_options = [5.0, 20.0, 40.0];
    for (var D, _pj_c = 0, _pj_a = time_options, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        D = _pj_a[_pj_c];
        cdd_immed_amt_for_D = [];
        for (var v, _pj_f = 0, _pj_d = amt_del_options, _pj_e = _pj_d.length; (_pj_f < _pj_e); _pj_f += 1) {
            v = _pj_d[_pj_f];
            cdd_immed_amt_for_D.push(round_to_closest_half((v / (1 + (kappa_value * D)))));
        }
        SE_trials["cdd_immed_amt"].push(...cdd_immed_amt_for_D);
        SE_trials["cdd_delay_amt"].push(...amt_del_options);
        SE_trials["cdd_delay_wait"].push(...value_repeat(D, amt_del_options.length));
    }
    SE_trials["category"] = value_repeat("SE_trials", SE_trials["cdd_immed_amt"].length);
    console.log(SE_trials);
    return SE_trials;
}
function max_neg_delta(kappa_value, amt_immed_options, amt_del_options, time_options) {
    /*
    Creates trials with the maximum negative delta subjective value (SV), that are not catch trials (Vdel > Vn).
    */
    var SV_diff, iSV_diff, ivl, ivn, min_SV_trials;
    min_SV_trials = {"category": [], "cdd_delay_wait": [], "cdd_immed_amt": [], "cdd_delay_amt": [], "deltaSV": []};
    for (var d, _pj_c = 0, _pj_a = time_options, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        d = _pj_a[_pj_c];
        SV_diff = 0;
        ivn = 0;
        ivl = 0;
        for (var vn, _pj_f = 0, _pj_d = amt_immed_options, _pj_e = _pj_d.length; (_pj_f < _pj_e); _pj_f += 1) {
            vn = _pj_d[_pj_f];
            for (var vl, _pj_i = 0, _pj_g = amt_del_options, _pj_h = _pj_g.length; (_pj_i < _pj_h); _pj_i += 1) {
                vl = _pj_g[_pj_i];
                if ((vl <= vn)) {
                    continue;
                }
                iSV_diff = ((vl / (1 + (kappa_value * d))) - vn);
                if ((iSV_diff < SV_diff)) {
                    SV_diff = iSV_diff;
                    [ivn, ivl] = [vn, vl];
                }
            }
        }
        min_SV_trials["category"].push("minSV");
        min_SV_trials["cdd_delay_wait"].push(d);
        min_SV_trials["cdd_immed_amt"].push(ivn);
        min_SV_trials["cdd_delay_amt"].push(ivl);
        min_SV_trials["deltaSV"].push(SV_diff);
    }
    return min_SV_trials;
}
</script>

In [ ]:
%%HTML
<script>
SE_trials = get_SE(kappa_value, amt_del_options, time_options);
SE_trials = add_SV(SE_trials, kappa_value);
//SE_trials = order_keys(SE_trials, keys_order);
min_trials = max_neg_delta(kappa_value, amt_immed_options, amt_del_options, time_options);
min_SV_trials = {};
var min_trials_keys = ['category', 'cdd_delay_amt', 'cdd_delay_wait', 'cdd_immed_amt'];
for (var key, _pj_c = 0, _pj_a = min_trials_keys, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
    key = _pj_a[_pj_c];
    min_SV_trials[key] = [];
    min_SV_trials[key].push(...min_trials[key]);
    min_SV_trials[key].push(...min_trials[key]);
}
min_SV_trials = add_SV(min_SV_trials, kappa_value);
//min_SV_trials = order_keys(min_SV_trials, keys_order);
</script>

In [ ]:
%%HTML
<script>
function target_SVdiff(min_SV_trials, Vmax, Amp) {
    /*
    Defines the target SV_deltas we want to sample given the SV range.
    */
    var SV_range_neg, SV_range_pos, min_sv_diff, sv_deltas;
    sv_deltas = [];
    for (var delta_sv, _pj_c = 0, _pj_a = min_SV_trials["deltaSV"], _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        delta_sv = _pj_a[_pj_c];
        sv_deltas.push(delta_sv);
    }
    min_sv_diff = Math.min(...sv_deltas);
    SV_range_pos = [];
    SV_range_neg = [];
    for (var g = 0, _pj_a = Number.parseInt((1 / Amp)); (g < _pj_a); g += 1) {
        if ((g !== 0)) {
            SV_range_pos.push(((Amp * Vmax) * g));
            SV_range_neg.push(((Amp * min_sv_diff) * g));
        }
    }
    return [SV_range_pos, SV_range_neg];
}
var tmp = target_SVdiff(min_SV_trials, V_max, Amp);
target_SV_deltas_pos = tmp[0]
target_SV_deltas_neg = tmp[1]
</script>

In [ ]:
%%HTML
<script>
function get_int_pos_SV_delta(target_SV_deltas, amt_immed_options, time_options, Vmax = V_max, valence = "pos") {
    var Vdel, Vimm, category, category_value, delta_sv, index, max_sv_trials;
    max_sv_trials = {"category": [], "cdd_delay_wait": [], "cdd_immed_amt": [], "cdd_delay_amt": []};
    Vimm = Math.min(...amt_immed_options);
    category = "Int+";
    if ((valence === "neg")) {
        Vimm = 10.0;
        category = "Int-";
    }
    for (var tmp, _pj_c = 0, _pj_a = mighty_numeration(target_SV_deltas, 1), _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        tmp = _pj_a[_pj_c];
        index = tmp[0];
        delta_sv = tmp[1];
        category_value = index;
        for (var D, _pj_f = 0, _pj_d = time_options, _pj_e = _pj_d.length; (_pj_f < _pj_e); _pj_f += 1) {
            D = _pj_d[_pj_f];
            Vdel = [round_to_closest_half(((delta_sv + Vimm) * (1 + (kappa_value * D))))];
            for (var v, _pj_i = 0, _pj_g = Vdel, _pj_h = _pj_g.length; (_pj_i < _pj_h); _pj_i += 1) {
                v = _pj_g[_pj_i];
                if ((v <= Vmax)) {
                    max_sv_trials["cdd_delay_amt"].push(v);
                    max_sv_trials["cdd_delay_wait"].push(D);
                    max_sv_trials["cdd_immed_amt"].push(Vimm);
                    max_sv_trials["category"].push((category + category_value.toString()));
                }
            }
        }
    }
    return max_sv_trials;
}
Int_pos = get_int_pos_SV_delta(target_SV_deltas_pos, amt_immed_options, time_options);
Int_pos = add_SV(Int_pos, kappa_value);
console.log(Int_pos)
//Int_pos = order_keys(Int_pos, keys_order);
</script>

In [ ]:
%%HTML
<script>
function get_int_neg_SV_delta(target_SV_deltas, amt_immed_options, time_options, Vmax, d0 = 180, valence = "pos") {
    var Vdel, Vimm, category, category_value, delta_sv, index, max_sv_trials;
    max_sv_trials = {"category": [], "cdd_delay_wait": [], "cdd_immed_amt": [], "cdd_delay_amt": []};
    Vimm = Math.max(...amt_immed_options);
    category = "Int+";
    if ((valence === "neg")) {
        Vdel = 65.0;
        category = "Int-";
    }
    for (var tmp, _pj_c = 0, _pj_a = mighty_numeration(target_SV_deltas, 1), _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        tmp = _pj_a[_pj_c];
        index = tmp[0];
        delta_sv = tmp[1];
        category_value = index;
        for (var D, _pj_f = 0, _pj_d = time_options, _pj_e = _pj_d.length; (_pj_f < _pj_e); _pj_f += 1) {
            D = _pj_d[_pj_f];
            if ((D > d0)) {
                continue;
            }
            Vimm = round_to_closest_half(((Vdel / (1 + (kappa_value * D))) - delta_sv));
            if ((Vimm > Vdel)) {
                continue;
            }
            max_sv_trials["cdd_delay_amt"].push(Vdel);
            max_sv_trials["cdd_delay_wait"].push(D);
            max_sv_trials["cdd_immed_amt"].push(Vimm);
            max_sv_trials["category"].push((category + category_value.toString()));
        }
    }
    return max_sv_trials;
}
Int_neg = get_int_neg_SV_delta(target_SV_deltas_neg, amt_immed_options, time_options, V_max, d0 = d0, valence = "neg");
Int_neg = add_SV(Int_neg, kappa_value);
//Int_neg = order_keys(Int_neg, keys_order);
console.log(Int_neg);
</script>

In [ ]:
%%HTML
<script>
function array_of_repetitions(value, repetitions) {
    var tmp = [];
    for (var i=0; i < repetitions; i++) {
        tmp.push(value);
    }
    return tmp;
}

function get_Max_SVdelta(time_options, Vmax = V_max, Vmin = V_min) {
    /*
    This function creates the trials with the Maximum Positive delta.
    By assigning the largest amount we can pay to the delayed options and the least we can pay to the immediate options
    */
    var Max_SVdelta;
    Max_SVdelta = {"category": ["MaxSV"], "cdd_delay_wait": [2], "cdd_immed_amt": [2.0], "cdd_delay_amt": [65.0]};
    Max_SVdelta["category"].push(...array_of_repetitions("MaxSV", time_options.length));
    Max_SVdelta["cdd_delay_amt"].push(...array_of_repetitions(Vmax, time_options.length));//([Vmax] * time_options.length));
    Max_SVdelta["cdd_immed_amt"].push(...array_of_repetitions(Vmin, time_options.length));
    Max_SVdelta["cdd_delay_wait"].push(...time_options);
    return Max_SVdelta;
}
Max_trials = get_Max_SVdelta(time_options, V_max, V_min);
console.log(Max_trials)
Max_SV_trials = {};
var max_trials_keys = ['category', 'cdd_delay_amt', 'cdd_delay_wait', 'cdd_immed_amt'];
for (var key, _pj_c = 0, _pj_a = max_trials_keys, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
    key = _pj_a[_pj_c];
    Max_SV_trials[key] = [];
    Max_SV_trials[key].push(...Max_trials[key]);
    Max_SV_trials[key].push(...Max_trials[key]);
}
Max_SV_trials = add_SV(Max_SV_trials, kappa_value);
//Max_SV_trials = order_keys(Max_SV_trials, keys_order);
console.log(Max_SV_trials)
</script>

In [ ]:
%%HTML
<script>
function dict_merge(list_dict) {
    var merged_dict, value;
    merged_dict = [];
    var keys_order = ["category", "cdd_immed_amt", "cdd_immed_wait", "cdd_delay_amt", "cdd_delay_wait", "delay_SV", "deltaSV"];
    for (var i = 0; i < list_dict.length; i++) {
        var dic = list_dict[i];
        for (var k = 0; k < keys_order.length; k++) {
            var key = keys_order[k];
            try {
                var tmp;
                if (dic.hasOwnProperty(key)) {
                    tmp = dic[key]
                }
                if (!merged_dict.hasOwnProperty(key)) {
                    merged_dict[key] = [];
                }
                if (tmp) {
                    merged_dict[key].push(...tmp);
                }
            } catch(error) {
                console.log(error);
            }
        }
    }
    
    return merged_dict;
}
trials = dict_merge([Max_SV_trials, Int_pos, SE_trials, Int_neg, min_SV_trials]);
trials["cdd_immed_wait"] = (value_repeat(0, trials["category"].length));
console.log(trials);
keys_order = ["category", "cdd_immed_amt", "cdd_immed_wait", "cdd_delay_amt", "cdd_delay_wait", "delay_SV", "deltaSV"];
//trials = order_keys(trials, keys_order);
</script>

In [ ]:
%%HTML
<script>
var _pj;
function _pj_snippets(container) {
    function in_es6(left, right) {
        if (((right instanceof Array) || ((typeof right) === "string"))) {
            return (right.indexOf(left) > (- 1));
        } else {
            if (((right instanceof Map) || (right instanceof Set) || (right instanceof WeakMap) || (right instanceof WeakSet))) {
                return right.has(left);
            } else {
                return (left in right);
            }
        }
    }
    container["in_es6"] = in_es6;
    return container;
}
_pj = {};
_pj_snippets(_pj);
function length_int_trials(trials, valence = "+") {
    var category, int_category, length_int_trials;
    category = [...trials["category"]];
    int_category = [];
    length_int_trials = 0;
    for (var c, _pj_c = 0, _pj_a = category, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        c = _pj_a[_pj_c];
        if (_pj.in_es6(valence, c)) {
            int_category.push(c);
            length_int_trials += 1;
        }
    }
    return length_int_trials;
}
function needed_categories(trials, valence = "+") {
    var category, int_category;
    category = [...(new Set(trials["category"]))];
    int_category = [];
    for (var c, _pj_c = 0, _pj_a = category, _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        c = _pj_a[_pj_c];
        if (_pj.in_es6(valence, c)) {
            int_category.push(c);
        }
    }
    return int_category;
}
function grab_additional_trial(trials, c, r) {
    var additional_trial, trial_category, trial_idx;
    for (var tmp, _pj_c = 0, _pj_a = mighty_numeration(trials["category"]), _pj_b = _pj_a.length; (_pj_c < _pj_b); _pj_c += 1) {
        tmp = _pj_a[_pj_c];
        trial_idx = tmp[0];
        trial_category = tmp[1];
        if ((trial_category === c)) {
            additional_trial = {"category": [(c.toString() + "_repeat")], 
                                "cdd_immed_amt": [trials["cdd_immed_amt"][trial_idx]], 
                                "cdd_immed_wait": [trials["cdd_immed_wait"][trial_idx]], 
                                "cdd_delay_amt": [trials["cdd_delay_amt"][trial_idx]], 
                                "cdd_delay_wait": [trials["cdd_delay_wait"][trial_idx]], 
                                "delay_SV": [trials["delay_SV"][trial_idx]], 
                                "deltaSV": [trials["deltaSV"][trial_idx]]};
            r = (r - 1);
            if ((r > 0)) {
                continue;
            }
            return additional_trial;
        }
    }
}
function add_trials(trials, trials_needed, valence = "+") {
    var additional_trial, int_category, nb_repeats;
    int_category = needed_categories(trials, valence);
    nb_repeats = 1;
    if ((int_category.length < trials_needed)) {
        nb_repeats = Number.parseInt((trials_needed / int_category.length));
    }
    for (var r = 0, _pj_a = nb_repeats; (r < _pj_a); r += 1) {
        for (var c, _pj_d = 0, _pj_b = int_category, _pj_c = _pj_b.length; (_pj_d < _pj_c); _pj_d += 1) {
            c = _pj_b[_pj_d];
            if ((trials_needed === 0)) {
                return trials;
            }
            additional_trial = grab_additional_trial(trials, c, r);
            if ((additional_trial === null)) {
                additional_trial = grab_additional_trial(trials, c, 1);
            }
            trials = dict_merge([trials, additional_trial]);
            trials_needed = (trials_needed - 1);
        }
    }
    return trials;
}
function standard_trial_number(trials, desired_trials) {
    var int_diff, length_neg, length_pos, trials_gap, trials_needed, trials_needed_neg, trials_needed_pos;
    trials_needed = (desired_trials - trials["category"].length);
    length_pos = length_int_trials(trials, valence = "+");
    length_neg = length_int_trials(trials, valence = "-");
    int_diff = (length_pos - length_neg);
    console.log((((((("(trials_needed,len_Int_pos,len_Int_neg) : (" + trials_needed) + ",") + length_pos) + ",") + length_neg) + ")"));
    if ((Math.abs(int_diff) < trials_needed)) {
        console.log("Were doomed");
        trials_gap = (trials_needed - Math.abs(int_diff));
        if ((int_diff < 0)) {
            trials_needed_pos = (trials_gap + ((trials_needed - trials_gap) / 2));
            trials = add_trials(trials, trials_needed_pos, valence = "+");
            trials_needed_neg = ((trials_needed - trials_gap) / 2);
            trials = add_trials(trials, trials_needed_neg, valence = "-");
        } else {
            trials_needed_neg = (trials_gap + ((trials_needed - trials_gap) / 2));
            trials = add_trials(trials, trials_needed_neg, valence = "-");
            trials_needed_pos = ((trials_needed - trials_gap) / 2);
            trials = add_trials(trials, trials_needed_pos, valence = "+");
        }
    } else {
        if ((int_diff > 0)) {
            trials = add_trials(trials, trials_needed, valence = "-");
        } else {
            trials = add_trials(trials, trials_needed, valence = "+");
        }
    }
    return trials;
}
desired_trials = 100;
trials = standard_trial_number(trials, desired_trials);
console.log(trials);
</script>